通过使用深度学习框架来简洁地实现 线性回归模型 生成数据集

In [18]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)#synthetic_data是人工数据合成的函数

调用框架中现有的API来读取数据

In [19]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个pytorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)#tensordataset:把输入的两类数据进行一一对应
    return data.DataLoader(dataset, batch_size, shuffle=is_train)#dataloader:重新排序后随机挑选出batch_size

batch_size = 10
data_iter = load_array((features, labels),batch_size)

next(iter(data_iter))

[tensor([[ 0.1102, -0.9346],
         [-0.7827, -0.2422],
         [ 0.5356,  1.4493],
         [-2.5595,  0.9294],
         [-0.0416, -0.2469],
         [ 0.0487, -1.1305],
         [-0.4907, -0.0422],
         [-0.8908, -1.4746],
         [-2.0097, -0.7458],
         [ 2.5660, -0.5310]]),
 tensor([[ 7.6025],
         [ 3.4612],
         [ 0.3331],
         [-4.0802],
         [ 4.9618],
         [ 8.1395],
         [ 3.3589],
         [ 7.4315],
         [ 2.7019],
         [11.1398]])]

使用框架的预定义好的层

In [20]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))#使用linear层，指定输入维度是2，输出维度是1

初始化模型参数

In [28]:
net[0].weight.data.normal_(0, 0.01)#weight访问到w,normal_使用正态分布换掉data
net[0].bias.data.fill_(0)#bias访问到b，fill_(0)初始化为0

tensor([0.])

计算均方误差使用的是MSELoss类，也称为平方 L2 范数

In [22]:
loss = nn.MSELoss()

实例化 SGD 实例

In [23]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)#存入模型中w,b等参数，学习率

训练过程代码与我们从零开始实现时所做的非常相似

In [29]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()#先将梯度清0
        l.backward()#已经自动sum过了，直接求梯度
        trainer.step()#调用这个函数进行模型的更新
    l = loss(net(features), labels)
    print(f'epoch{epoch + 1}, loss{l:f}')

epoch1, loss0.000184
epoch2, loss0.000097
epoch3, loss0.000097


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [30]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0013,  0.0001])
b的估计误差： tensor([0.0002])
